##  Project Objective
The objective of this data engineering project is to build a scalable, end-to-end data pipeline that ingests, processes, and enriches Bitcoin’s daily market data. This pipeline will serve as the foundation for analytical reporting, time-series feature engineering, and predictive modeling related to Bitcoin's market behavior.

# Benefits
A Bitcoin investment advisory firm
- The advisory team can trust the data for research, reports, and investor recommendations without manual wrangling or risk of errors.
- Helps the firm detect trends, shifts, and patterns in the Bitcoin market — empowering analysts to make better buy/sell/hold calls.
- Enables risk analysts to flag abnormal behavior early, protecting clients and improving the firm's credibility.
- Enhances the firm’s client reporting and supports data-driven decision-making.
- Reduces operational overhead, freeing up analysts to focus on strategy instead of data prep.

##  Description
This notebook handles the ingestion of raw Bitcoin daily market data. It reads the dataset from a CSV source, infers schema, performs initial inspection, and writes the output to a raw (bronze) layer for further processing.

##  Tools & Technologies Used in This Notebook
| Tool / Library | Purpose |
|----------------|---------|
| **Apache Spark (PySpark)** | Distributed data processing for ingesting and handling large datasets |
| **Databricks / Jupyter Notebook** | Interactive environment for writing and testing code |
| **Delta Lake / Parquet** | Optimized storage format for raw (bronze) layer |
| **DBFS or Cloud Storage (e.g., `/mnt/raw/`)** | Storage location for input dataset |

In [0]:
#%fs ls /Volumes/bitcoin/market_data/coin/Bitcoin_history_data.csv    # List the contents of the specified directory in DBFS (Databricks File System)


path,name,size,modificationTime
dbfs:/Volumes/bitcoin/market_data/coin/Bitcoin_history_data.csv,Bitcoin_history_data.csv,335981,1753691038000


In [0]:
# Read the Bitcoin history CSV file into a Spark DataFrame
# - header=True: uses the first row as column names
# - inferSchema=True: automatically infers data types for each column
df_raw = spark.read.csv("/Volumes/bitcoin/market_data/coin/Bitcoin_history_data.csv", header=True, inferSchema=True)

# Display the schema of the DataFrame to understand the structure and data types
df_raw.printSchema()

# Show the first 5 rows of the DataFrame for a quick preview of the data
df_raw.show(5)

# Write the raw DataFrame to a Delta Lake table in the 'bronze' layer
# - format("delta"): saves the data in Delta format for ACID transactions and versioning
# - mode("overwrite"): replaces any existing data in the target table
# - saveAsTable(): registers the table in the metastore under the specified name
df_raw.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("bitcoin.market_data.bronze_bitcoin")


In [0]:
import dlt
from pyspark.sql.functions import *

@dlt.table(
    name="bronze_bitcoin_historical",
    comment="Raw Bitcoin historical data ingested from CSV into the bronze layer"
)
def ingest_bitcoin_data():
    # Read the Bitcoin CSV file from DBFS
    df_raw = spark.read.csv(
        "/Volumes/bitcoin/market_data/coin/Bitcoin_history_data.csv",
        header=True,
        inferSchema=True
    )
    
    return df_raw


The Delta Live Tables (DLT) module is not supported on this cluster.
 You should either create a new pipeline or use an existing pipeline to run DLT code.

The Delta Live Tables (DLT) module is not supported on this cluster.
 You should either create a new pipeline or use an existing pipeline to run DLT code.

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-6740942322275571>, line 1
----> 1 import dlt
      2 from pyspark.sql.functions import *
      4 @dlt.table(
      5     name="bronze_bitcoin",
      6     comment="Raw Bitcoin historical data ingested from CSV into the bronze layer"
      7 )
      8 def ingest_bitcoin_data():
      9     # Read the Bitcoin CSV file from DBFS

File /databricks/python_shell/lib/dbruntime/autoreload/discoverability/hook.py:71, in AutoreloadDiscoverabilityHook._patched_import(self, name, *args, **kwargs)
     65 if not self._should_hint and (
     66     (module := sys.modules.get(absolute_name)) is not None and
     67     (fname := get_allowed_file_name_or_none(module)) is not None and
     68     (mtime := os.stat(fname).st_mtime) > self.last_mtime_by_modname.get(
     69         absolute_name, float("inf")) and not self._should_hint):
  